In [ ]:
# import necessary libraries
# virtual environment is tied to python 3.10.2
import math
import numpy as np
import sympy as sm
sm.init_printing(use_latex='mathjax')
import matplotlib.pyplot as plt
import matplotlib.patches as pch
%matplotlib inline
from matplotlib.ticker import MaxNLocator # to make integer ticks on x axis

In [ ]:
# define variables
t           = sm.symbols("t", real=True)                    # variable of integration
n           = sm.symbols("n", integer=True, positive=True)  # n-th harmonic
tau         = sm.symbols("tau", real=True)                  # piece-wise definition boundary
duty        = sm.symbols("duty")                            # duty cycle
amplitude   = sm.symbols("amplitude", real=True)
freq        = sm.symbols("freq", real=True)

In [ ]:
# define target function
# f(t) = 2t+1 if 0 <= t < tau and 0 if tau <= t < (2*tau)
# T = 2*tau since duty cycle is set to 50%
# define target function as a fourier series approximation
fn = a_o + a_n + b_n
fn = fn.simplify()

In [ ]:
# plot the sum of all the terms
fapprox = sum(terms)
sm.plot(fapprox,(t,-1,1),title="Finite Fourier Series Approximation");